In [223]:
from data_handle import DatasetHandler
from model_handle import ModelHandler

#train_data_path = '../data/kddcup2010_train.txt'
test_data_path = '../data/kddcup2010_test.txt'

## Evaluation
- batch : batch(user) by 49+1(excercise, prediction)

In [224]:
from constants import *

data_handler = DatasetHandler()
#train_generator = data_handler.get_data_generator(train_data_path)
val_generator = data_handler.get_data_generator(test_data_path)

model_handler = ModelHandler(input_dim, hidden_dim, num_layers, output_dim)
model_handler.load_model('/home/fanzhilin/code/konwledge_tracing/1-py_dkt-master/outputs/20_0.3381_0.7372.pth')
model_handler.compile_model()

loading data:    : 538it [00:00, 1396.08it/s]

------------------------------
---------- cuda:0 ----------
------------------------------
compile model --------------------
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

lossFunc(
  (loss_fn): BCELoss()
)

DKT(
  (lstm): LSTM(1322, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=661, bias=True)
  (sig): Sigmoid()
)
------------------------------


In [225]:
# model_handler.evaluate(val_generator)

## Predict

In [226]:
# test sample 
batch_sample = next(iter(val_generator))
batch_sample.shape

torch.Size([128, 50, 1322])

In [227]:
import random
rand_idx = random.randint(0, batch_sample.shape[0] - 1)
rand_idx#随机取一个样本

83

In [228]:
x = batch_sample[rand_idx]
x.shape#取出一个样本，代表一个学生的做题记录

torch.Size([50, 1322])

In [229]:
model_handler.predict(x)

q_sequence tensor([570., 274., 124., 105., 323.,  71., 533., 522., 145., 573., 123.,  77.,
        281., 195., 422., 589., 536., 312., 556., 330., 283., 169., 507., 537.,
        370., 125., 107., 170.,  21.,  18., 224., 355., 658., 275., 429., 500.,
        591., 115., 647., 464., 173.,  83., 298., 314.,  49., 103., 622., 443.,
        567., 223.])
a_sequence tensor([1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.],
       device='cuda:0')
--------------------------------------------------
sol excercise tags: 
 tensor([570., 274., 124., 105., 323.,  71., 533., 522., 145., 573., 123.,  77.,
        281., 195., 422., 589., 536., 312., 556., 330., 283., 169., 507., 537.,
        370., 125., 107., 170.,  21.,  18., 224., 355., 658., 275., 429., 500.,
        591., 115., 647., 464., 173.,  83., 298., 314.,  49., 103., 622., 4

In [230]:
import random

rand_idx = random.randint(0, batch_sample.shape[0] - 1)
x = batch_sample[rand_idx]
prob = model_handler.predict(x)

q_sequence tensor([468., 332., 468., 332.,  40., 438.,  75., 421., 380., 421.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.])
a_sequence tensor([1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
--------------------------------------------------
sol excercise tags: 
 tensor([468., 332., 468., 332.,  40., 438.,  75., 421.])
result excercise tags: 
 tensor([1., 0., 1., 1., 1., 1., 1., 1.], device='cuda:0')
--------------------------------------------------
predict excercise tag 421.0
ground truth : 1.0
this student has a 97.65% chance of solving this problem


In [231]:
x.shape#从batch中取出一个样本，代表一个学生的做题记录

torch.Size([50, 1322])

In [232]:
model_handler.model

DKT(
  (lstm): LSTM(1322, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=661, bias=True)
  (sig): Sigmoid()
)

In [233]:
#获得模型预测的概率，即学生下一次做对的概率
def _cal_prob(x,model):#x=[1, seq_len, numofq*2]

    # qt
    delta = x[:,:,:QUESTION_NUM] + x[:,:,QUESTION_NUM:]

    # qt+1
    delta = delta[:,1:,:].permute(0,2,1)

    # yt
    pred = model(x)
    y = pred[:, :MAX_SEQ - 1,:]

    # pred at+1
    temp = torch.matmul(y, delta) # 1, MAX_SEQ, MAX_SEQ-1(prob)

    # get excercise prob from diagonal matrix
    prob = torch.diagonal(temp, dim1=1, dim2=2) # 1, MAX_SEQ-1(prob)\

    return prob.squeeze(0)#再把batch维度去掉 1, MAX_SEQ-1(prob) -> MAX_SEQ-1(prob) 每一个值代表预测该问题上学生作答正确的概率

In [234]:
x.nonzero()#x只有46个非零元素1，代表学生做了46道题，因为要跳过第一个时间步，所以只有45道题，为什么要跳过第一个时间步？因为第一个时间步的数据是用来预测第二个时间步的数据，所以要跳过第一个时间步

tensor([[  0, 468],
        [  1, 993],
        [  2, 468],
        [  3, 332],
        [  4,  40],
        [  5, 438],
        [  6,  75],
        [  7, 421],
        [  8, 380],
        [  9, 421]])

In [235]:
# 一条预测的序列
import torch
if len(x.size()) == 2:#先给x增加一个维度，因为要送入模型·
    x = x.unsqueeze(0)
print(x.shape)#x的维度是(1, MAX_SEQ, numofq*2)

x = x.to(model_handler.model.device)

prob = _cal_prob(x, model_handler.model)
print(prob.shape)#prob的维度是(MAX_SEQ-1,)

q_sequence = _get_q_sequence(x)
print(q_sequence.shape)#q_sequence的维度是(MAX_SEQ,)

a_sequence = _get_a_sequence(x)
print(a_sequence.shape)#a_sequence的维度是(MAX_SEQ,)

if -1 in q_sequence:#如果有-1，说明这个step学生的作答是填充了的
    last_excercise_tag = torch.nonzero(q_sequence == -1)[0][0].item() - 1 #torch.nonzero(q_sequence == -1)寻找-1的位置
else:#如果没有-1
    last_excercise_tag = len(q_sequence) - 1#为什么要-1？？


# print("q_sequence",q_sequence)
# print("a_sequence",a_sequence)


print('-' * 50)
print(f'单个学生有效作答sol excercise tags: \n {q_sequence[:last_excercise_tag]}')
print(f'对应的原始结果result excercise tags: \n {a_sequence[:last_excercise_tag]}')
print('-' * 50)

print(f'predict excercise tag {q_sequence[last_excercise_tag]}')
print(f'ground truth : {a_sequence[last_excercise_tag]}')
print(f'this student has a {prob[last_excercise_tag-1]*100:.2f}% chance of solving this problem')
print('-' * 50)
print(f'predict prob {prob}')




torch.Size([1, 50, 1322])
torch.Size([49])


torch.Size([50])
torch.Size([50])
--------------------------------------------------
单个学生有效作答sol excercise tags: 
 tensor([468., 332., 468., 332.,  40., 438.,  75., 421., 380.])
对应的原始结果result excercise tags: 
 tensor([1., 0., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
--------------------------------------------------
predict excercise tag 421.0
ground truth : 1.0
this student has a 97.65% chance of solving this problem
--------------------------------------------------
predict prob tensor([0.9643, 0.9357, 0.9774, 0.9996, 0.1447, 0.9999, 0.9584, 0.9573, 0.9765,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SqueezeBackward1>)


In [236]:
prob#只有45个值，因为用户只做了45道题（跳过第一个时间步）

tensor([0.9643, 0.9357, 0.9774, 0.9996, 0.1447, 0.9999, 0.9584, 0.9573, 0.9765,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SqueezeBackward1>)

In [237]:
#这里目前有单看不太懂!
def _get_q_sequence(q_seq_one_hot):#q_seq_one_hot=[1, seq_len, numofq*2]

    q_sequence = []
    one_hot_excercise_tags = q_seq_one_hot[:, :, :QUESTION_NUM] + q_seq_one_hot[:, :, QUESTION_NUM:]#q_seq_one_hot=[1, seq_len, numofq]学生做题标记，无论正错都是1
    one_hot_excercise_tags = one_hot_excercise_tags.squeeze(0)#去掉batch维度 形状变为(seq_len, numofq)
    
    for one_hot_excercise_tag in one_hot_excercise_tags:#对每一个step处理
        
        try:#找到用户作答最后一次的位置
            excercise_tag = torch.nonzero(one_hot_excercise_tag).item()#找到学生作答的位置索引，其实就是题目编号
        except:
            excercise_tag = -1 #如果找不到作答记录，说明学生在这个step时间上没有作答，标记为-1

        q_sequence.append(excercise_tag)#学生原始的做题记录，保证是maxstep长度，如果没有做题，标记为-1

    return torch.Tensor(q_sequence)#list返回tensor类型，方便操作

def _get_a_sequence(q_seq_one_hot):#q_seq_one_hot=[1, seq_len, numofq*2]
    q_seq_one_hot = q_seq_one_hot.squeeze(0)#去掉batch维度 形状变为(seq_len, numofq*2)
    a_sequence = ((q_seq_one_hot[:, :QUESTION_NUM] - q_seq_one_hot[:, QUESTION_NUM:]).sum(1) + 1) // 2 #拿到原始学生作答对应的正误记录
    return a_sequence#对于没有作答的step，标记为0 ，而不是-1

In [238]:
q_seq_one_hot=x
q_sequence = []
one_hot_excercise_tags = q_seq_one_hot[:, :, :QUESTION_NUM] + q_seq_one_hot[:, :, QUESTION_NUM:]#q_seq_one_hot=[1, seq_len, numofq]学生做题标记，无论正错都是1
one_hot_excercise_tags = one_hot_excercise_tags.squeeze(0)#去掉batch维度 形状变为(seq_len, numofq)
one_hot_excercise_tags.shape
one_hot_excercise_tags.sum(1)#每一行的和都是1，但学生只做了46道题，所以只有46个1，其他都是0

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [239]:
q_sequence = _get_q_sequence(x)

In [240]:
q_sequence

tensor([468., 332., 468., 332.,  40., 438.,  75., 421., 380., 421.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.])

In [241]:
q_sequence == -1

tensor([False, False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [254]:
if -1 in q_sequence:#如果有-1，说明这个step学生的作答是填充了的,
    last_excercise_tag = torch.nonzero(q_sequence == -1)[0][0].item() - 1 #torch.nonzero(q_sequence == -1)寻找-1的位置，[0][0].item()取第一次出现的位置
else:#如果没有-1
    last_excercise_tag = len(q_sequence) - 1
last_excercise_tag

9

In [257]:
torch.nonzero(q_sequence == -1)[0][0].item(),q_sequence[9]

(10, tensor(421.))

In [258]:
q_sequence

tensor([468., 332., 468., 332.,  40., 438.,  75., 421., 380., 421.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.])

In [260]:
#用户当前作答
q_sequence[:last_excercise_tag-1]

tensor([468., 332., 468., 332.,  40., 438.,  75., 421.])

In [262]:
#需要预测的问题id 为什么不是380？
q_sequence[last_excercise_tag]

tensor(421.)

In [244]:
q_sequence[torch.nonzero(q_sequence == -1)[0][0].item()]

tensor(-1.)

In [245]:
if -1 in q_sequence:#如果有-1，说明这个step学生的作答是填充了的,
    last_excercise_tag = torch.nonzero(q_sequence == -1)[0][0].item() - 1 #torch.nonzero(q_sequence == -1)寻找-1的位置，[0][0].item()取第一次出现的位置
else:#如果没有-1
    last_excercise_tag = len(q_sequence) - 1

In [246]:
for one_hot_excercise_tag in one_hot_excercise_tags:#对每一个step的one_hot_excercise_tag进行处理
    print(one_hot_excercise_tag)
    print(torch.nonzero(one_hot_excercise_tag).item())
    break

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [247]:
for one_hot_excercise_tag in one_hot_excercise_tags:
    try:
        excercise_tag = torch.nonzero(one_hot_excercise_tag).item()
    except:
        excercise_tag = -1

    q_sequence.append(excercise_tag)


AttributeError: 'Tensor' object has no attribute 'append'